In [4]:
# import statements
import numpy as np
from path import Planner

In [9]:
maps = {} # dictionary of maps, a map is a dictionary of coordinates and their corresponding values - one map for each layer

# instantiate map
def instantiate_map():
    map = {}
    for i in np.arange(0, 379.4, 0.1):
        for j in np.arange(0, 189.7, 0.1):
            map[(round(i, 1), round(j, 1))] = None
    return map

In [10]:
# linear interpolation
# returns a list of points between two points

def get_points(x1, x2, y1, y2, step_size = 0.1):
    if x1 == x2:
        y_points = np.round(np.append(np.arange(y1, y2, step_size, dtype='float64'), y2), 1)
        x_points = np.round(np.full((np.size(y_points),), x1), 1)
        x_points = np.round(np.full((np.size(y_points),), x1), 1)

        assert np.size(x_points) == np.size(y_points)
        return x_points, y_points
    
    x_points = np.round(np.append(np.arange(x1, x2, step_size, dtype='float64'), x2), 1)
    y_points = np.zeros(np.size(x_points), dtype='float64')
    
    for i in range(np.size(x_points)):
        y_points[i] = round(((y2-y1)/(x2-x1))*(x_points[i]-x1) + y1, 1)
    
    assert np.size(x_points) == np.size(y_points)
    return x_points, y_points

In [11]:
def update_maps():
    path = Planner('sample_gcode/test_big_box.gcode')
    x_start = y_start = None # start point of the path set to None
    
    for coord in path.generate():
        x, y, z, command = coord
        
        # round coordinates to 1 decimal place
        x = round(x, 1)
        y = round(y, 1)
        
        # move coordinates to the left from (0, 0)
        x = x - 5.2
        y = y - 5.2
        
        if z not in maps:
            # create a new map for this z value
            maps[z] = instantiate_map()
            x_start = y_start = None
            
        # for the first point in the path
        if x_start is None and y_start is None:
            x_start = x
            y_start = y
        
        # linear interpolation of points between the start and end points
        x_points, y_points = get_points(x_start, x, y_start, y)
        
        # update the start point
        x_start = x
        y_start = y
        
        # update the map for both arms
        for x_point, y_point in zip(x_points, y_points):
            x_point = round(x_point, 1)
            y_point = round(y_point, 1)
            x_point_2 = round(x_point + 189.7, 1)
            maps[z][(x_point, y_point)] = ("arm_1", command)
            maps[z][(x_point_2, y_point)] = ("arm_2", command)

In [12]:
# clear all maps
def clear_maps():
    for map in maps:
        for key in map:
            map[key] = None

In [13]:
# clear all maps and update them
clear_maps()
update_maps()

In [ ]:
# save maps to file
import pickle

count = 0
for map in maps:
    # write dictionary to pkl file
    with open('maps/map_' + str(count) + '.pkl', 'wb') as fp:
        pickle.dump(maps[map], fp)
        print(f"dictionary {count} saved successfully to file")
    count+=1

In [ ]:
print(maps[0.3][(379.3, 189.6)])
print(maps[0.3][(132.1, 4.2)])

print(maps[3.0][(379.3, 189.6)])
print(maps[3.0][(132.1, 4.2)])

In [12]:
arm_1_cardinal_maps = {}
arm_2_cardinal_maps = {}

In [15]:
def generate_cardinal_maps():
    path = Planner('sample_gcode/test_big_box.gcode')
    x_start = y_start = None # start point of the path set to None
    
    for coord in path.generate():
        x, y, z, command = coord
        
        # round coordinates to 1 decimal place
        x = round(x, 1)
        y = round(y, 1)
        
        # move coordinates to the left from (0, 0)
        x = x - 5.2
        y = y - 5.2
        
        if z not in arm_1_cardinal_maps and z not in arm_2_cardinal_maps:
            # create a new map for this z value
            arm_1_cardinal_maps[z] = {}
            arm_2_cardinal_maps[z] = {}
            x_start = y_start = None
            
        # for the first point in the path
        if x_start is None and y_start is None:
            x_start = x
            y_start = y
        
        # linear interpolation of points between the start and end points
        x_points, y_points = get_points(x_start, x, y_start, y)
        
        # update the start point
        x_start = x
        y_start = y
        
        count = 0
        # update the map for both arms
        for x_point, y_point in zip(x_points, y_points):
            x_point = round(x_point, 1)
            y_point = round(y_point, 1)
            x_point_2 = round(x_point + 189.7, 1)
            arm_1_cardinal_maps[z][count] = (x_point, y_point)
            arm_2_cardinal_maps[z][count] = (x_point_2, y_point)
            count+=1

In [13]:
def clear_cardinal_maps():
    for map in arm_1_cardinal_maps:
        del arm_1_cardinal_maps[map]
    for map in arm_2_cardinal_maps:
        del arm_2_cardinal_maps[map]

In [16]:
clear_cardinal_maps()
generate_cardinal_maps()

In [17]:
print(arm_1_cardinal_maps[0.3])

{0: (1.0, 188.2), 1: (1.1, 187.8), 2: (1.2, 187.9), 3: (1.3, 188.0), 4: (1.4, 188.1), 5: (1.5, 188.2), 6: (1.6, 188.3), 7: (1.7, 188.4), 8: (1.8, 188.5), 9: (1.9, 188.6), 10: (2.0, 187.5), 11: (2.1, 187.6), 12: (2.2, 187.7), 13: (2.3, 187.8), 14: (2.4, 187.9), 15: (2.5, 188.0), 16: (2.6, 188.1), 17: (2.7, 188.2), 18: (2.8, 188.3), 19: (2.9, 188.4), 20: (3.0, 188.5), 21: (3.1, 188.6), 22: (3.1, 188.6), 23: (3.3, 187.7), 24: (3.4, 187.8), 25: (3.5, 187.9), 26: (3.6, 188.0), 27: (3.7, 188.1), 28: (3.8, 188.2), 29: (3.9, 188.3), 30: (4.0, 188.4), 31: (4.1, 188.5), 32: (4.2, 188.6), 33: (4.3, 187.6), 34: (4.4, 187.7), 35: (4.5, 187.8), 36: (4.6, 187.9), 37: (4.7, 188.0), 38: (4.8, 188.1), 39: (4.9, 188.2), 40: (5.0, 188.3), 41: (5.1, 188.4), 42: (5.2, 188.5), 43: (5.3, 188.6), 44: (5.4, 187.5), 45: (5.5, 187.6), 46: (5.6, 187.7), 47: (5.7, 187.8), 48: (5.8, 187.9), 49: (5.9, 188.0), 50: (6.0, 188.1), 51: (6.1, 188.2), 52: (6.2, 188.3), 53: (6.3, 188.4), 54: (6.4, 188.5), 55: (6.5, 188.6), 5

In [ ]:
# save cardinal_maps to file
import pickle

count = 0
for map in arm_1_cardinal_maps:
    # write dictionary to pkl file
    with open('cardinal_maps/arm1_cardinal_map_' + str(count) + '.pkl', 'wb') as fp:
        pickle.dump(arm_1_cardinal_maps[map], fp)
        print(f"dictionary {count} saved successfully to file")
        count+=1

count = 0
for map in arm_2_cardinal_maps:
    with open('cardinal_maps/arm2_cardinal_map_' + str(count) + '.pkl', 'wb') as fp:
        pickle.dump(arm_2_cardinal_maps[map], fp)
        print(f"dictionary {count} saved successfully to file")
    count+=1

In [20]:
# extract maps from file
import pickle

new_cardinal_maps = {}
count = 0

while True:
    try:f
        # read dictionary from pkl file
        with open('cardinal_maps/arm1_cardinal_map_' + str(count) + '.pkl', 'rb') as fp:
            new_cardinal_maps[count] = pickle.load(fp)
            print(f"dictionary {count} loaded successfully from file")
        count += 1
    except FileNotFoundError:
        break

dictionary 0 loaded successfully from file
dictionary 1 loaded successfully from file
dictionary 2 loaded successfully from file
dictionary 3 loaded successfully from file
dictionary 4 loaded successfully from file
dictionary 5 loaded successfully from file
dictionary 6 loaded successfully from file
dictionary 7 loaded successfully from file
dictionary 8 loaded successfully from file
dictionary 9 loaded successfully from file
dictionary 10 loaded successfully from file
dictionary 11 loaded successfully from file
dictionary 12 loaded successfully from file
dictionary 13 loaded successfully from file
dictionary 14 loaded successfully from file
dictionary 15 loaded successfully from file
dictionary 16 loaded successfully from file
dictionary 17 loaded successfully from file
dictionary 18 loaded successfully from file


In [21]:
new_cardinal_maps[0] == arm_1_cardinal_maps[0.3]

True

In [ ]:
# extract maps from file
import pickle

maps = {}
count = 0

while True:
    try:
        # read dictionary from pkl file
        with open('maps/map_' + str(count) + '.pkl', 'rb') as fp:
            maps[count] = pickle.load(fp)
            print(f"dictionary {count} loaded successfully from file")
        count += 1
    except FileNotFoundError:
        break